<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-20 20:18:42--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12074 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.79K  --.-KB/s    in 0s      

2022-06-20 20:18:42 (121 MB/s) - ‘utils.py’ saved [12074/12074]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [8]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [11]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [ ]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        validation_data=val_ds,
                        class_weight=class_weight)

In [ ]:
rgb_model.layers[0].trainable = True
rgb_creator.re_compile(rgb_model)

In [ ]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=5,
                        validation_data=val_ds,
                        class_weight=class_weight)

In [ ]:
!cp .logs/checkpoints/RGB-model-05.h5 ./drive/MyDrive/checkpoints/rgb-model-5-ft.h5

In [12]:
rgb_model = keras.models.load_model('./drive/MyDrive/checkpoints/rgb-model-5-ft.h5',
                                    custom_objects={'KerasLayer':hub.KerasLayer})

In [16]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        initial_epoch=5,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 6/10
519/519 [==============================] - 1958s 4s/step - loss: 4.3639 - auc: 0.9605 - accuracy: 0.9003 - val_loss: 0.9830 - val_auc: 0.8415 - val_accuracy: 0.7928
Epoch 7/10
519/519 [==============================] - 409s 787ms/step - loss: 3.8970 - auc: 0.9675 - accuracy: 0.9138 - val_loss: 1.3638 - val_auc: 0.7679 - val_accuracy: 0.7081
Epoch 8/10
519/519 [==============================] - 415s 797ms/step - loss: 2.7486 - auc: 0.9802 - accuracy: 0.9366 - val_loss: 0.8088 - val_auc: 0.8590 - val_accuracy: 0.8007
Epoch 9/10
519/519 [==============================] - 413s 794ms/step - loss: 1.8688 - auc: 0.9871 - accuracy: 0.9581 - val_loss: 1.1744 - val_auc: 0.7976 - val_accuracy: 0.7908
Epoch 10/10
519/519 [==============================] - 415s 798ms/step - loss: 1.3974 - auc: 0.9897 - accuracy: 0.9593 - val_loss: 1.0319 - val_auc: 0.8305 - val_accuracy: 0.7905


In [18]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        initial_epoch=10,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 11/15
519/519 [==============================] - 409s 786ms/step - loss: 1.7362 - auc: 0.9865 - accuracy: 0.9564 - val_loss: 0.9274 - val_auc: 0.8580 - val_accuracy: 0.8158
Epoch 12/15
519/519 [==============================] - 415s 797ms/step - loss: 1.6086 - auc: 0.9916 - accuracy: 0.9639 - val_loss: 0.9341 - val_auc: 0.8480 - val_accuracy: 0.8119
Epoch 13/15
519/519 [==============================] - 413s 793ms/step - loss: 0.9780 - auc: 0.9946 - accuracy: 0.9762 - val_loss: 0.9531 - val_auc: 0.8568 - val_accuracy: 0.8070
Epoch 14/15
519/519 [==============================] - 415s 797ms/step - loss: 0.9299 - auc: 0.9951 - accuracy: 0.9783 - val_loss: 1.0175 - val_auc: 0.8576 - val_accuracy: 0.8221
Epoch 15/15
519/519 [==============================] - 417s 800ms/step - loss: 0.6745 - auc: 0.9975 - accuracy: 0.9853 - val_loss: 1.2467 - val_auc: 0.8296 - val_accuracy: 0.8173


In [19]:
!cp .logs/checkpoints/RGB-model-14.h5 ./drive/MyDrive/checkpoints/rgb-model-14-ft.h5

cp: cannot stat '.logs/checkpoints/RGB-model-14.h5': No such file or directory
